# Debug

This script contains issues that have been found during the development of this project. Here some basic calculation will be done to locate and solve the issues

## depth_image_converter

### Description

This package perform the transformation from object point cloud to a depth image, the general logic is:
1. Transform the point cloud (in panda_link0 frame) into camera_color_optical_frame;
2. Project the points onto the pixel frame of the depth image

The first step is varified to be correct,

![image](/home/franka/ws_perception/src/vision/doc/pics/for_ipynb/point_cloud_2_frames.png)


but during the 2nd step, the projected depth image is wrong, the image is like following 

![image](/home/franka/ws_perception/src/vision/doc/pics/for_ipynb/bug_depth_image.png)

during the projection /camera/aligned_depth_to_color/camera_info is used as K, which may caused the issue.

So here we load the there K's provided by the camera and do some tests:

In [26]:
import numpy as np

K_aligned = np.array([[910, 0,  641], 
                      [ 0, 909, 367], 
                      [ 0,  0,   1 ]])

K_depth = np.array([[910,  0,  641],
                    [ 0 , 909, 367],
                    [ 0 ,  0,   1 ]])

K_color = np.array([[428, 0,  429],
                    [ 0, 428, 236],
                    [ 0,  0,   1 ]])

Now, we introduce a pre-defined poit, which is at [0, 0, 0.5] w.r.t. camera_color_optical frame and calculate the pixel frame of it

In [28]:
test_point = np.array([0.7, 0, 1])

test_point_aligned_hom = K_aligned.dot(test_point)
test_point_aligned = np.array([test_point_aligned_hom[0]/test_point_aligned_hom[2], 
                               test_point_aligned_hom[1]/test_point_aligned_hom[2]])

test_point_color_hom = K_color.dot(test_point)
test_point_color = np.array([test_point_color_hom[0]/test_point_color_hom[2],
                             test_point_color_hom[1]/test_point_color_hom[2]])

print("Using K_aligned, the pixel position is: ")
print(test_point_aligned)

print("")

print("Using K_color, the pixel position is: ")
print(test_point_color)

Using K_aligned, the pixel position is: 
[1278.  367.]

Using K_color, the pixel position is: 
[728.6 236. ]


### Issue Solved

The issue is located at the usage of cv::Mat::at() function, at line 118 there is:

```c++ 
depth_mat.at<double>(curr_point_pixel[1], curr_point_pixel[0]) = std::round(objects_cloud_cam_->at(i).z * std::pow(10, 3));
```

but in <> tag after "at", we should specify the data type according to the image encoding, but not the data type of input image, for CV_16UC1 it should be ushort, so after correction:

```c++ 
depth_mat.at<ushort>(curr_point_pixel[1], curr_point_pixel[0]) = std::round(objects_cloud_cam_->at(i).z * std::pow(10, 3));
```